In [1]:
import gradio as gr
from transformers import pipeline
import time

exo = pipeline('text-generation',model='./models/checkpoint-90000', tokenizer='gpt2', config={'max_length':1000})
machina = lambda text: exo(text)[0]['generated_text']

In [22]:
# TODO make a generator class to store previous outputs, settings, etc

last_output = ""

def clean_gen(prompt,N=3):
    outputs = []
    # extract last sentance of input
    clean_prompt = prompt#.split('.')[-1] # TODO careful it may pick up on numbers
    clean_prompt.replace('\n','')
    clean_prompt = clean_prompt.strip()

    for i in range(N):
        #print(clean_prompt)
        outputs.append(machina(clean_prompt).replace(clean_prompt,f'{i+1})'))
    return outputs

def generate(prompt,radio):
    global last_output
    if prompt[-1] == '\n':
        outputs = clean_gen(prompt)
        last_output = "\n\n".join(outputs)
    if radio != None:
        # TODO query prompt class
        pass
    print(radio)
    return last_output
    

demo = gr.Interface(
    fn=generate,
    inputs=[
        gr.Textbox(lines=2, value="Name Here..."),
        gr.Radio(["1","2","3"]),
    ],
    
    outputs="text",
    live=True
)
demo.launch()

/Users/kpearson/anaconda3/envs/unity_ml/lib/python3.8/site-packages/gradio/blocks.py:155: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fe1d10ed4c0>, 'http://127.0.0.1:7872/', None)

Traceback (most recent call last):
  File "/Users/kpearson/anaconda3/envs/unity_ml/lib/python3.8/site-packages/gradio/routes.py", line 298, in run_predict
    output = await app.blocks.process_api(
  File "/Users/kpearson/anaconda3/envs/unity_ml/lib/python3.8/site-packages/gradio/blocks.py", line 790, in process_api
    result = await self.call_function(fn_index, inputs, iterator)
  File "/Users/kpearson/anaconda3/envs/unity_ml/lib/python3.8/site-packages/gradio/blocks.py", line 697, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/kpearson/anaconda3/envs/unity_ml/lib/python3.8/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/kpearson/anaconda3/envs/unity_ml/lib/python3.8/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/kpearson/anaconda3/envs/unity_ml/lib/python3.8/site-packages/anyio/_backends/_asynci